# 变量之间的相关性分析实践

变量相关性分析旨在量化两个变量之间关系的强弱程度。常用的方法包括**皮尔逊相关系数**、**点二列相关系数**、**偏相关系数**、**斯皮尔曼相关系数**和**肯德尔秩相关系数**。



## 皮尔逊相关系数

本节首先通过示例数据演示如何手动计算和使用 Python 库函数验证皮尔逊相关系数：

$$r_{XY} = \frac{\sum_{i=1}^{n}(X_i-\bar{X})(Y_i-\bar{Y})}{\sqrt{\sum_{i=1}^{n}(X_i-\bar{X})^2}\sqrt{\sum_{i=1}^{n}(Y_i-\bar{Y})^2}}$$



In [6]:
import numpy as np
from scipy import stats

# 示例数据：身高（米）与体重（kg）
X = np.array([1.70, 1.80, 1.75, 1.68, 1.82])
Y = np.array([65, 78, 72, 66, 74])

# 数据中心化
X_centered = X - X.mean()
Y_centered = Y - Y.mean()

# 手动计算皮尔逊相关系数
r_manual = np.sum(X_centered * Y_centered) / np.sqrt(np.sum(X_centered**2) * np.sum(Y_centered**2))
print(f"皮尔逊相关系数（手动计算）: {r_manual:.4f}")

# 使用 scipy 库计算并验证结果
r_scipy, p_value = stats.pearsonr(X, Y)
print(f"皮尔逊相关系数（scipy）: {r_scipy:.4f}")
print(f"P 值: {p_value:.4f}")

皮尔逊相关系数（手动计算）: 0.9080
皮尔逊相关系数（scipy）: 0.9080
P 值: 0.0331


## 点二列相关系数
当其中一个变量为二分类变量（0或1）时，我们采用**点二列相关系数**进行分析
$$r_{pb} = \frac{\bar{X}_1 - \bar{X}_0}{s_X}\sqrt{\frac{n_1 n_0}{n^2}}$$

In [7]:
import numpy as np
from scipy import stats

# 示例数据：game 为二分类变量（是否玩游戏），score 为连续变量（考试成绩）
game = np.array([1, 0, 1, 0, 1])  
score = np.array([65, 95, 70, 90, 75])  

# 样本量和分组情况
n1, n0 = np.sum(game == 1), np.sum(game == 0)
n = len(game)

# 计算各组平均成绩
mean1, mean0 = score[game == 1].mean(), score[game == 0].mean()

# 标准差使用总体标准差
s_x = np.std(score, ddof=0)

# 点二列相关系数手动计算
r_pb_manual = (mean1 - mean0) / s_x * np.sqrt(n1 * n0 / n**2)
print(f"点二列相关系数（手动计算）: {r_pb_manual:.4f}")

# scipy库验证
r_pb, p_value = stats.pointbiserialr(game, score)
print(f"点二列相关系数（scipy）: {r_pb:.4f}")
print(f"P 值: {p_value:.4f}")

点二列相关系数（手动计算）: -0.9522
点二列相关系数（scipy）: -0.9522
P 值: 0.0124


## 偏相关系数：
但实际研究中，我们经常需要控制其他变量的影响，这时需要用到**偏相关系数**：

$$r_{XY.Z} = \frac{r_{XY} - r_{XZ}r_{YZ}}{\sqrt{(1-r_{XZ}^2)(1-r_{YZ}^2)}}$$

In [8]:
import numpy as np
import pandas as pd
from scipy import stats
import pingouin as pg

# 示例数据：气温可能同时影响冰淇淋销量和游泳人数
temperature = np.array([22, 25, 28, 31, 34, 37, 40])
ice_cream = np.array([12, 18, 21, 26, 28, 35, 38])
swimming = np.array([8, 12, 16, 21, 23, 27, 30])

# 计算简单相关系数
r_xy = stats.pearsonr(ice_cream, swimming)[0]
r_xz = stats.pearsonr(ice_cream, temperature)[0]
r_yz = stats.pearsonr(swimming, temperature)[0]

# 手动计算偏相关系数（控制temperature）
r_xy_z = (r_xy - r_xz * r_yz) / np.sqrt((1 - r_xz**2) * (1 - r_yz**2))
print(f"简单相关系数: {r_xy:.4f}")
print(f"偏相关系数（控制气温后）: {r_xy_z:.4f}")

# 使用 pingouin 库验证偏相关系数
df = pd.DataFrame({'ice_cream': ice_cream, 'swimming': swimming, 'temperature': temperature})
pcorr = pg.partial_corr(data=df, x='ice_cream', y='swimming', covar='temperature')
print(f"偏相关系数（pingouin）: {pcorr['r'].values[0]:.4f}")
print(f"P 值: {pcorr['p-val'].values[0]:.4f}")

简单相关系数: 0.9939
偏相关系数（控制气温后）: 0.3744
偏相关系数（pingouin）: 0.3744
P 值: 0.4647


## 斯皮尔曼相关系数

适用于数据不满足正态分布假设或存在明显异常值的情形：

$$
r_s = 1 - \frac{6\sum_{i=1}^n d_i^2}{n(n^2-1)}
$$

In [9]:
import numpy as np
from scipy import stats

# 示例数据（带波动）
X = np.array([100, 200, 500, 300, 150])  # 刷题量
Y = np.array([85, 88, 91, 87, 86])       # 考试分数

rank_X = stats.rankdata(X)
rank_Y = stats.rankdata(Y)

# 计算排名差的平方和
d_squared = np.sum((rank_X - rank_Y)**2)
n = len(X)
r_s_manual = 1 - (6 * d_squared) / (n * (n**2 - 1))
print(f"斯皮尔曼相关系数（手动计算）: {r_s_manual:.4f}")

# 使用 scipy 验证
r_s, p_value = stats.spearmanr(X, Y)
print(f"斯皮尔曼相关系数（scipy）: {r_s:.4f}")
print(f"P 值: {p_value:.4f}")

斯皮尔曼相关系数（手动计算）: 0.9000
斯皮尔曼相关系数（scipy）: 0.9000
P 值: 0.0374


## 肯德尔秩相关系数

肯德尔秩相关系数关注变量排名之间的一致性，特别适用于较小样本：

$$
\tau = \frac{n_c - n_d}{\frac{1}{2}n(n-1)}
$$

其中，$n_c$为协调对数，$n_d$为不协调对数。

In [10]:
import numpy as np
from scipy import stats
# 示例数据（非完美正相关）
X = np.array([2, 4, 8, 6, 3])    # 刷题时长
Y = np.array([65, 75, 85, 72, 78])  # 考试分数

# 手动计算协调对 / 不协调对
n = len(X)
n_c = 0
n_d = 0
for i in range(n):
    for j in range(i + 1, n):
        if (X[j] - X[i]) * (Y[j] - Y[i]) > 0:
            n_c += 1  # 协调对
        elif (X[j] - X[i]) * (Y[j] - Y[i]) < 0:
            n_d += 1  # 不协调对

# 计算 Kendall's tau
tau_manual = (n_c - n_d) / (0.5 * n * (n - 1))
print(f"协调对数量: {n_c}")
print(f"不协调对数量: {n_d}")
print(f"肯德尔相关系数（手动计算）: {tau_manual:.4f}")

# 使用 scipy 验证
tau, p_value = stats.kendalltau(X, Y)
print(f"肯德尔相关系数（scipy）: {tau:.4f}")
print(f"P 值: {p_value:.4f}")

协调对数量: 7
不协调对数量: 3
肯德尔相关系数（手动计算）: 0.4000
肯德尔相关系数（scipy）: 0.4000
P 值: 0.4833
